In [7]:
import gym
from gym.core import ObservationWrapper
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import display
from tqdm.notebook import tqdm

from models.agentRandomSearch import AgentRandomSearch
from models.agentSarsa import AgentSarsa
from models.agentQlearning import AgentQlearning
from models.agentQlearning_ER import AgentQlearning_ER

# configurations
numTargetFrame=1000
numSucceed=10 # number of epochs it reaches the target
numEpoch=int(1e6)
numRecordStep=int(1e3)

agent_choice='qlearning_er' # choose from 'random_search', 'sarsa', 'qlearning', 'qlearning_er'
alpha=0.05 # for 'sarsa', 'qlearning', 'qlearning_er' only
gamma=0.9 # for 'sarsa', 'qlearning', 'qlearning_er' only
epsilon_max=1.0 # for 'sarsa', 'qlearning', 'qlearning_er' only
epsilon_min=0.3 # for 'sarsa', 'qlearning', 'qlearning_er' only
epsilon_halflife=200 # for 'sarsa', 'qlearning', 'qlearning_er' only
memory_size=1000000 # for 'qlearning_er' only 
memory_sampling=10 # for 'qlearning_er' only 

buckets=[5, 5, 20, 20]
# buckets=[1, 10, 20, 20]

In [8]:
# changes the continuous state space to discrete ones
class DiscretizeStateWrapper(gym.ObservationWrapper):
    def __init__(self, env, buckets=[1, 1, 6, 12], upper_bounds=[], lower_bounds=[]):
        super(ObservationWrapper, self).__init__(env)
        self.buckets = buckets
        if upper_bounds==[]:
#             self.upper_bounds = [env.observation_space.high[0], 0.5, env.observation_space.high[2], math.radians(50) / 1.]
            self.upper_bounds = [env.observation_space.high[0]/2, 4, env.observation_space.high[2]/2, math.radians(180) / 1.]
        else:
            self.upper_bounds = upper_bounds            
        if lower_bounds==[]:
#             self.lower_bounds = [env.observation_space.low[0], -0.5, env.observation_space.low[2], -math.radians(50) / 1.]
            self.lower_bounds = [env.observation_space.low[0]/2, -4, env.observation_space.low[2]/2, -math.radians(180) / 1.]
        else:
            self.lower_bounds = lower_bounds
    def observation(self, obs):
        discretized = list()
        for i in range(len(obs)):
            scaling = (obs[i] + abs(self.lower_bounds[i])) / (self.upper_bounds[i] - self.lower_bounds[i])
            new_obs = int(round((self.buckets[i] - 1) * scaling))
            new_obs = min(self.buckets[i] - 1, max(0, new_obs)) # VW: needed?
            discretized.append(new_obs)
        return tuple(discretized)

# # doesn't do anything
# class RewardWrapper(gym.RewardWrapper):
#     def __init__(self, env):
#         super().__init__(env)
    
#     def reward(self, rew):
#         # modify rew
#         return rew

# # doesn't do anything
# class ActionWrapper(gym.ActionWrapper):
#     def __init__(self, env):
#         super().__init__(env)
    
#     def action(self, act):
#         # modify act
#         return act
    
env = DiscretizeStateWrapper(gym.make('CartPole-v1'),buckets)


In [9]:
def check_if_state_exist(self, state):
    if state3 not in self.q_table:
        self.q_table[state]=np.zeros(env.action_space.n)


In [10]:
# # A naive agent that goes to right or left depending on the tilt angle
# class AgentBasic(object):
#     def get_action(self, states):
#         return 0 if states[2] < 0 else 1

In [11]:
def run_episode(env, agent):  
    """Runs the env for a certain amount of steps with the given parameters. Returns the reward obtained"""
    state = env.reset()
    totalreward = 0
    for _ in range(numTargetFrame):
        state, reward, done = agent.train(state)   
        totalreward += reward
        if done:
            break
    return totalreward

In [ ]:
epoch_list=[]
reward_list=[]
reward_microlist=[]
bestreward = 0
succeed = 0

if(agent_choice=='random_search'):
    agent=AgentRandomSearch(env)
elif(agent_choice=='sarsa'):
    agent=AgentSarsa(env, alpha=alpha, gamma=gamma, epsilon_max=epsilon_max, epsilon_min=epsilon_min, epsilon_halflife=epsilon_halflife)
elif(agent_choice=='qlearning'):
    agent=AgentQlearning(env, alpha=alpha, gamma=gamma, epsilon_max=epsilon_max, epsilon_min=epsilon_min, epsilon_halflife=epsilon_halflife)
elif(agent_choice=='qlearning_er'):
    agent=AgentQlearning_ER(env, alpha=alpha, gamma=gamma, epsilon_max=epsilon_max, epsilon_min=epsilon_min, epsilon_halflife=epsilon_halflife, memory_size=100000, memory_sampling=10)

if(agent_choice=='sarsa' or agent_choice=='qlearning' or agent_choice=='qlearning_er'):
    agent.train_mode(True)
    
with tqdm(total=numSucceed, desc='Number of success', position=0) as succeedbar:
    for i in tqdm(range(numEpoch), desc='Epoch', position=1):  
        reward = run_episode(env, agent)
        reward_microlist.append(reward)
        if reward > bestreward:
            bestreward = reward
            # considered solved if the agent lasts for the required number of timesteps
        if reward == numTargetFrame:
            succeed+=1
            succeedbar.update(1)
            if(succeed==numSucceed):
                break
        elif(i%numRecordStep==0):
            reward_list.append(np.mean(reward_microlist))
            epoch_list.append(i)
            reward_microlist=[]
        
if (succeed==numSucceed):
    print("Finished running and solution found in epoch # {}! =D \n".format(i)) # first epoch starts from label 0
elif (succeed!=0):
    print("Finished running and didn't meet requirement, but solution was found {} times. =) \n".format(numSucceed)) # first epoch starts from label 0
else:
    print("Finished running but solution not found. =\\ \n")
print("The best reward was {} steps.".format(bestreward))
        
print("#################################")
print("#                               #")
print("#        Done training!!        #")
print("#                               #")
print("#################################")

In [ ]:
plt.plot(epoch_list[1:], reward_list[1:])
plt.xlabel("Epoch")
plt.ylabel("Reward")
plt.ylim([0,np.amax(reward_list[1:])*1.2])
plt.savefig('reward_vs_epoch_{}.png'.format(agent_choice))
del epoch_list, reward_list

In [ ]:
def show_episode(env):  
    """ Records the frames of the environment obtained using the given parameters... Returns RGB frames"""
    state = env.reset()
    firstframe = env.render(mode='rgb_array')
    frames = [firstframe]
    
    if(agent_choice=='sarsa' or agent_choice=='qlearning' or agent_choice=='qlearning_er'):
        agent.train_mode(False)
    for _ in range(numTargetFrame):
        action = agent.get_action(state)
        state, reward, done, info = env.step(action)
        frame = env.render(mode='rgb_array')
        frames.append(frame)
        if done:
            break
    return frames

def display_frames_as_gif(frames, filename_gif = None):
    """
    Displays a list of frames as a gif, with controls
    """
    plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 36)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50, repeat=False)
    if filename_gif: 
        print("Saving animation...")
        anim.save(filename_gif, writer = 'pillow', fps=10)
        print("Animation saves as gif at: {}".format(filename_gif))
        
frames = show_episode(env)
display_frames_as_gif(frames, filename_gif="random_search_play_{}.gif".format(agent_choice))
env.close()
print("###############################")
print("#                             #")
print("#        Done saving!!        #")
print("#                             #")
print("###############################")